In [1]:
"""
Written by Matteo Dunnhofer - 2017

Test procedure - evaluating accuracy on validation (test) set
"""
import os.path
import tensorflow as tf
import train_util as tu
from models import alexnet
import numpy as np
import pandas as pd 

TOP_K = 5
K_PATCHES = 5
DISPLAY_STEP = 10
IMAGENET_PATH = '/scratch/gpfs/qlu/ILSVRC2012'
CKPT_PATH = '/tigress/qlu/logs/ILSVRC2012/ckpt-alexnet00'


top_k = TOP_K
k_patches = K_PATCHES
display_step = DISPLAY_STEP
imagenet_path = IMAGENET_PATH
ckpt_path = CKPT_PATH

/home/qlu/.conda/envs/brainiak/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# def test(top_k, k_patches, display_step,imagenet_path,ckpt_path):
test_images = sorted(os.listdir(os.path.join(imagenet_path, 'ILSVRC2012_img_val')))
test_labels = tu.read_test_labels(os.path.join(imagenet_path, 'data/ILSVRC2012_validation_ground_truth.txt'))
test_examples = len(test_images)

x = tf.placeholder(tf.float32, [None, 224, 224, 3])
y = tf.placeholder(tf.float32, [None, 1000])

_, pred = alexnet.classifier(x, 1.0)

# calculate the average precision of the crops of the image
avg_prediction = tf.div(tf.reduce_sum(pred, 0), k_patches)

# accuracy
top1_correct = tf.equal(tf.argmax(avg_prediction, 0), tf.argmax(y, 1))
top1_accuracy = tf.reduce_mean(tf.cast(top1_correct, tf.float32))

topk_correct = tf.nn.in_top_k(tf.stack([avg_prediction]), tf.argmax(y, 1), k=top_k)
topk_accuracy = tf.reduce_mean(tf.cast(topk_correct, tf.float32))

saver = tf.train.Saver()

In [10]:
type(test_labels[0])

numpy.ndarray

In [3]:
sess = tf.InteractiveSession()

In [5]:
# tf.get_default_graph().as_graph_def()
# for op in sess.graph.get_operations(): 
#     print(op.name)

In [4]:
saver.restore(sess, os.path.join(ckpt_path, 'alexnet-cnn.ckpt'))

INFO:tensorflow:Restoring parameters from /tigress/qlu/logs/ILSVRC2012/ckpt-alexnet00/alexnet-cnn.ckpt


In [5]:
i = 0 
image_patches = tu.read_k_patches(os.path.join(imagenet_path, 'ILSVRC2012_img_val', test_images[0]), k_patches)
label = test_labels[0]
conv1 = tf.get_default_graph().get_tensor_by_name('alexnet_cnn/alexnet_cnn_conv1/Relu:0')
conv2 = tf.get_default_graph().get_tensor_by_name('alexnet_cnn/alexnet_cnn_conv2/Relu:0')
conv3 = tf.get_default_graph().get_tensor_by_name('alexnet_cnn/alexnet_cnn_conv3/Relu:0')
conv4 = tf.get_default_graph().get_tensor_by_name('alexnet_cnn/alexnet_cnn_conv4/Relu:0')
conv5 = tf.get_default_graph().get_tensor_by_name('alexnet_cnn/alexnet_cnn_conv5/Relu:0')
fc1 = tf.get_default_graph().get_tensor_by_name('alexnet_classifier/alexnet_classifier_fc1/Relu:0')
fc2 = tf.get_default_graph().get_tensor_by_name('alexnet_classifier/alexnet_classifier_fc2/Relu:0')
fc3 = tf.get_default_graph().get_tensor_by_name('alexnet_classifier/alexnet_classifier_output/Add:0')

fetch_list = [top1_accuracy, topk_accuracy, conv1, conv2, conv3, conv4, conv5, fc1, fc2, fc3]
fetched_list = sess.run(fetch_list, feed_dict={x: image_patches, y: [label]})
[top1_f, topk_f, conv1_f, conv2_f, conv3_f, conv4_f, conv5_f, fc1_f, fc2_f, fc3_f] = fetched_list

In [13]:
print(np.shape(conv1_f))
print(np.shape(conv2_f))
print(np.shape(conv3_f))
print(np.shape(conv4_f))
print(np.shape(conv5_f))
print(np.shape(fc1_f))
print(np.shape(fc2_f))
print(np.shape(fc3_f))

(5, 56, 56, 96)
(5, 27, 27, 256)
(5, 13, 13, 384)
(5, 13, 13, 384)
(5, 13, 13, 256)
(5, 4096)
(5, 4096)
(5, 1000)


In [6]:
with tf.Session(config=tf.ConfigProto()) as sess:
    saver.restore(sess, os.path.join(ckpt_path, 'alexnet-cnn.ckpt'))

    total_top1_accuracy = 0.
    total_topk_accuracy = 0.

    for i in range(test_examples):
        # taking a few patches from an image
        image_patches = tu.read_k_patches(os.path.join(imagenet_path, 'ILSVRC2012_img_val', test_images[i]), k_patches)
        label = test_labels[i]

        top1_a, topk_a = sess.run([top1_accuracy, topk_accuracy], feed_dict={x: image_patches, y: [label]})
        total_top1_accuracy += top1_a
        total_topk_accuracy += topk_a
#         if i % display_step == 0:
#             print ('Examples done: {:5d}/{} ---- Top-1: {:.4f} -- Top-{}: {:.4f}'.format(
#                 i + 1, test_examples, total_top1_accuracy / (i + 1), top_k, total_topk_accuracy / (i + 1)))

    print ('---- Final accuracy ----')
    print ('Top-1: {:.4f} -- Top-{}: {:.4f}'.format(total_top1_accuracy / test_examples, top_k, total_topk_accuracy / test_examples))
    print ('Top-1 error rate: {:.4f} -- Top-{} error rate: {:.4f}'.format(1 - (total_top1_accuracy / test_examples), top_k, 1 - (total_topk_accuracy / test_examples)))

In [4]:
        print ('---- Final accuracy ----')
        print ('Top-1: {:.4f} -- Top-{}: {:.4f}'.format(total_top1_accuracy / test_examples, top_k, total_topk_accuracy / test_examples))
        print ('Top-1 error rate: {:.4f} -- Top-{} error rate: {:.4f}'.format(1 - (total_top1_accuracy / test_examples), top_k, 1 - (total_topk_accuracy / test_examples)))

---- Final accuracy ----
Top-1: 0.0000 -- Top-5: 0.0000
Top-1 error rate: 1.0000 -- Top-5 error rate: 1.0000
